In [1]:
#set up shitty model with jeopardy questions - idea being that i can replace the corpus later 
import pandas as pd #might want to use it
import numpy as np #working with arrays/matrices after all! 
import gensim # Word embeddings
import random #random choice for bot's first word
from sklearn.metrics.pairwise import cosine_similarity

Thoughts: How many times does a word appear? The computer should only choose the most common words. It can't repeat any of the user_input or the bot_input.

Miles: look up most common english words in a word2vec model, use the top 20,000 words? 10,000? https://wordcounter.io/blog/how-many-words-does-the-average-person-know/   

Distance: This could be helpful: https://stackoverflow.com/questions/32446703/find-closest-vector-from-a-list-of-vectors-python

# Using GloVe

In [2]:
#using numpy 
import numpy as np
def load_glove_model(file):
    print("Loading...")
    f = open(file,'r')
    model = {}
    for line in f:
        whole_line = line.split()
        words = whole_line[0]
        embedding = np.array([float(val) for val in whole_line[1:]])
        model[words] = embedding
    print("Done.",len(model)," words loaded!")
    return model
#help from https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python

In [5]:
model = load_glove_model('glove.6B/glove.6B.300d.txt')

Loading...
Done. 400000  words loaded!


# Common English Words

Create a list of common english words from https://www.ef.edu/english-resources/english-vocabulary/top-3000-words/ with help from https://www.reddit.com/r/learnpython/comments/1zcbi4/is_there_a_way_to_get_a_list_of_common_english/

In [3]:
with open("common_words.txt") as f:
    cwordlist = []
    for line in f:
        cwordlist.append(line.strip().lower().replace('-',''))

In [6]:
common_word_vectors = {}
unvectorized_common_words = []
for word in cwordlist:
    if word in model.keys():
        common_word_vectors[word] = model[word]
    else:
        unvectorized_common_words.append(word)

In [63]:
#reshape the dictionary arrays 
common_word_vectors['truth'].shape

for key, value in common_word_vectors.items():
    common_word_vectors[key] = value.reshape(1,-1)
    
for key, value in model.items():
    model[key] = value.reshape(1,-1)

common_word_vectors['truth'].shape

(1, 300)

# Create the Game's Function

In [ ]:
#function inputs
#example input: 
user_input='string'
user_dict = {0: user_input}
bot_dict = {0: random.choice(cwordlist)}
user_word_list = [user_input]
bot_word_list = []
#could explore ordered dictionaries: https://stackoverflow.com/questions/16125229/last-key-in-python-dictionary

Eventual UI notes: 
- The user_response is going to be ignored until the end of the function. At the end of the function, we'll compare the user_response word with the bot_response word, and if they are the same then we'll have convergence. 
- The bot_response is the last word the bot said. -1 on the list
- The user_response is the last word the user said. -1 on the user_word_list
- The user_word_list[-2] is the word the user said LAST round. 
- The computer will say, "I was thinking of new_bot_response. You said user_response[-1], so we got convergence/we still need to converge- what's the convergence of new_bot_response and user_repsonse[-1]?"  

In [130]:
user_word_list = []
bot_word_list = [] 

In [131]:
def converge(user_input='str'):
    """Return a word that (hopefully, eventually) matches the user's next input word."""
    user_word_list.append(user_input)
    #on the first round, bot_word_list is empty:
    if len(bot_word_list) == 0: 
        bot_response = random.choice(cwordlist)
        bot_word_list.append(bot_response)
        print (f"You said {user_word_list[-1]}, and I was thinking of {bot_response}! Let's try and converge.")
        print (bot_word_list)
    #on the next rounds, the bot_word_list is longer 
    else: 
        mean_vector = (model[user_word_list[-2]] #-1 is user_input, but we want to match the previous user_input
                       + 
                       model[bot_word_list[-1]])/2
        #can add a clause here for no-repeats tomorrow! 
        for word, vector in common_word_vectors.items():
            cos_sim_dict[float(cosine_similarity(mean_vector, vector))] = word
        max_cos_sim = max(cos_sim_dict.keys())
        bot_response = cos_sim_dict[max_cos_sim]
        bot_word_list.append(bot_response)
        if bot_response == user_input:
            print (f":) You said {user_input}, and I was thinking of {bot_response}! CONVERGENCE!!!!! :) ")
        else: 
            print (f"You said {user_input}, but I was thinking of {bot_response}." 
                   f" What's the convergence of {user_input} and {bot_response}?")
            print (bot_word_list)

In [132]:
converge('banana')

You said banana, and I was thinking of expression! Let's try and converge.
['expression']


In [133]:
converge('happy')

You said happy, but I was thinking of king. What's the convergence of happy and king?
['expression', 'king']
